In [11]:
import pandas as pd
import numpy as np
import os
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from konlpy.tag import Komoran

data_dir = '../analysis_files/files/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# 1. 300_TRANSCRIPT_KL.csv부터 492_TRANSCRIPT_KL.csv까지 파일이 있으면 train_data로 불러옵니다.
train_data = pd.DataFrame()  # 빈 데이터프레임 생성
for i in range(300, 493):
    filename = f"{i}_TRANSCRIPT_KL.csv"
    try:
        data = pd.read_csv(os.path.join(data_dir, filename), encoding='cp949')
        train_data = pd.concat([train_data, data], ignore_index=True)
    except FileNotFoundError:
        continue
    
train_data

,Unnamed: 0,speaker,speaker_index,value,translate
0,0,Ellie,0,hi i'm ellie thanks for coming in today,안녕 나는 엘리야 오늘 와줘서 고마워요
1,1,Ellie,0,i was created to talk to people in a safe and ...,저는 안전하고 안전한 환경에 있는 사람들과 대화하기 위해 만들어졌습니다
2,2,Ellie,0,think of me as a friend i don't judge i can't ...,저를 친구로 생각해 주세요. 저는 컴퓨터라고 판단하지 않습니다
3,3,Ellie,0,i'm here to learn about people and would love ...,저는 사람들에 대해 배우기 위해 여기에 왔고 당신에 대해 배우고 싶습니다
4,4,Ellie,0,i'll ask a few questions to get us started and...,"시작하기 위해 몇 가지 질문을 할 것이며, 답변이 완전히 기밀인 것이 있으면 언제든..."
...,...,...,...,...,...
47125,290,Ellie,0,asked_everything (okay i think i have asked ev...,모든 것을 물었습니다. (알겠습니다. 제가 필요한 모든 것을 요청했다고 생각합니다
47126,291,Ellie,0,appreciate_open (thanks for sharing your thoug...,감사합니다_open (당신의 생각을 저와 공유해 주셔서 감사합니다)
47127,292,Ellie,0,bye (goodbye),안녕 (계속)
47128,293,Participant,1,bye and thank you,안녕 그리고 감사합니다


In [8]:
# 2. train_data['speaker_index']가 1인 값의 train_data['translate'] 값을 TfidfVectorizer를 사용하여 단어의 tf-idf 값을 계산합니다.
speaker_1_data = train_data[train_data['speaker_index'] == 1]  # speaker_index가 1인 데이터 추출
speaker_1_data['translate'].fillna('', inplace=True)  # NaN 값을 빈 문자열로 처리
speaker_1_data

C:\Users\user\AppData\Local\Temp\ipykernel_4276\2097317361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speaker_1_data['translate'].fillna('', inplace=True)  # NaN 값을 빈 문자열로 처리


,Unnamed: 0,speaker,speaker_index,value,translate
6,6,Participant,1,good,좋아요.
9,9,Participant,1,atlanta georgia,애틀랜타 조지아 주
12,12,Participant,1,um my parents are from here um,저희 부모님은 여기 출신입니다
14,14,Participant,1,i love it,좋아요'.
16,16,Participant,1,i like the weather,나는 날씨를 좋아합니다
...,...,...,...,...,...
47122,287,Participant,1,um,음.
47123,288,Participant,1,i am very proud of the fact that,저는 그 사실이 매우 자랑스럽습니다
47124,289,Participant,1,uh i don't give up,어 나는 포기하지 않습니다
47128,293,Participant,1,bye and thank you,안녕 그리고 감사합니다


In [12]:
# 문장 전처리 함수 (형태소 분석 + 단어 통합)
def preprocess_sentence(sentence):
    komoran = Komoran()
    pos = komoran.pos(sentence) # 형태소 분석
    processed_sentence = ""
    for word, tag in pos:
        if tag in ['NN', 'NNG', 'NNP', 'VV', 'VA']: # 명사와 동사만 고려 (다른 품사는 제외)
            processed_sentence += word + " "
        return processed_sentence.strip()

speaker_1_data['preprocessing'] = speaker_1_data['translate'].apply(preprocess_sentence) # 문장 전처리 적용

AttributeError: Java package 'kr.co.shineware.nlp.komoran.core' is not valid

In [ ]:
sentences = [sentence.split() for sentence in speaker_1_data['preprocessing']]
word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=1, sg=1) # Word2Vec 모델 학습

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(speaker_1_data['preprocessing'])

In [ ]:
features = vectorizer.get_feature_names()
for i, sentence in enumerate(speaker_1_data['preprocessing']):
    tfidf_row = tfidf_matrix[i].toarray()[0]
    sorted_indices = tfidf_row.argsort()[::-1] # tf-idf 값으로 정렬된 인덱스
    top_keywords = [features[idx] for idx in sorted_indices[:20]] # 상위 20개 단어 추출
    print(f"Sentence: {sentence}")
    print(f"Word Embedding: {word2vec_model[sentence]}")
    print(f"Top Keywords: {top_keywords}")
    print()

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(speaker_1_data['translate'])
tfidf_matrix

# 2. train_data['speaker_index']가 1인 값의 train_data['translate'] 값을 TfidfVectorizer를 사용하여 단어의 tf-idf 값을 계산합니다.
speaker_1_data = train_data[train_data['speaker_index'] == 1]  # speaker_index가 1인 데이터 추출
speaker_1_data['translate'].fillna('', inplace=True)  # NaN 값을 빈 문자열로 처리

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(speaker_1_data['translate'])
tfidf_matrix

In [ ]:
# 3. train_data['translate']에서 주요 단어들을 추출하여 상위 20개의 단어들과 tf-idf 값을 출력합니다.
features = vectorizer.get_feature_names_out()
tfidf_values = tfidf_matrix.toarray()

for i in range(len(speaker_1_data)):
    tfidf_row = tfidf_values[i]
    sorted_indices = tfidf_row.argsort()[::-1]  # tf-idf 값으로 정렬된 인덱스
    top_keywords = [features[idx] for idx in sorted_indices[:20]]  # 상위 20개 단어 추출
    tfidf_scores = tfidf_row[sorted_indices[:20]]  # 상위 20개 단어의 tf-idf 값 추출

print(f"Top Keywords:")
for keyword, tfidf_score in zip(top_keywords, tfidf_scores):
    print(f"Keyword: {keyword}, TF-IDF Score: {tfidf_score}")
print()